In [1]:
from nanorsm_parallel import *

Kafka server not available, Databroker will not be able to upload datum


In [2]:
# load elemental image data, align it with pystackreg and generate a tranform matrix
scan = '346726-346736'
interval = 2
parent_path = '//data//users//2025Q2//Marschilok_2025Q2//'
data_path = f"{parent_path}xrf//"
output_path = f"{parent_path}nanoRSM//{scan}_{interval}//"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created directory: {output_path}")


sid_list = get_sid_list([scan],interval)
elem = 'Ni_K'
file_list = [
    f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
    for sid in sid_list
]
im_stack = load_ims(file_list)
num_frame,im_row,im_col = np.shape(im_stack)
im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg
im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly
slider_view(im_stack_test)


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
# if there are more elements to be aligned and stacked
elem_list = ['Ni_K', 'Mn_K','Co_K']
stack = []
for i, elem in enumerate(elem_list):
    file_list = [
        f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
        for sid in sid_list
    ]
    im_stack = load_ims(file_list)
    im_stack_test = interp_sub_pix(im_stack, trans_matrix)
    imp = np.sum(im_stack_test, axis=0)
    stack.append(imp[np.newaxis, ...])
    tifffile.imwrite(f"{output_path}{elem}.tiff",im_stack_test.astype(np.float32),imagej=True)
    print(f"save {elem} to {output_path}{elem}.tiff")
stack = np.concatenate(stack, axis=0)                  

Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Ni_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Ni_K.tiff


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Mn_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Mn_K.tiff


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

save Co_K to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//Co_K.tiff


In [4]:
# sum all diffraction patterns from local files to determine roi
det_name = 'merlin1'

tot = sum_all_h5_data_db_parallel(sid_list, det = det_name)
roi = select_roi(np.log(tot+1))

Using 10 workers (desired: 10, available cores: 40).


Processing Scans:   0%|          | 0/6 [00:00<?, ?it/s]

Selected ROI: [14, 73, 92, 105]


In [5]:
# save all parameters to a json file for future reference

scan_row = im_row
scan_col = im_col
mon_name = 'sclr1_ch4'
threshold = None
roi_offset = [0,0]
data_store = 'reduced' # this will reduce the data stored. If use 'full' it can be over 100G
microscope = 'mll' # choose 'mll' or 'zp'

params_db = read_params_db(sid_list,microscope=microscope,det=det_name)
params_user = {"scan ids": sid_list,
               "fluorescence data path": data_path,
               "output path": output_path,
               "element list": elem_list,
               "element for alignment": elem,
               "alignment matrix": trans_matrix,
               "scan dimensions": [scan_row,scan_col],
               "detector name": det_name,
               "detector roi": roi,
               "threshold": threshold,
               "monitor": mon_name,
               "roi offset": roi_offset,
               "data store": data_store
              }
params = params_db | params_user

if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created directory: {output_path}")
    
params = convert_numpy(params)

with open(f"{output_path}parameters.json", "w") as f:
    json.dump(params, f, indent=4)
    print(f"write parameters to {output_path}parameters.json")


write parameters to //data//users//2025Q2//Marschilok_2025Q2//nanoRSM//346726-346736_2//parameters.json


In [6]:
# load diffraction patterns and align them

diff_data = load_h5_data_db_parallel(sid_list,det=det_name,mon=mon_name,roi=roi,threshold=threshold)
# diff_data = nanorsm.load_h5_files_parallel(files,roi)
sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)


/nsls2/conda/envs/2025-2.0-py311-tiled/lib/python3.11/site-packages/pymongo/synchronous/mongo_client.py:2554: UserWarning: MongoClient opened before fork. May not be entirely fork-safe, proceed with caution. See PyMongo's documentation for details: https://www.mongodb.com/docs/languages/python/pymongo-driver/current/faq/#is-pymongo-fork-safe-
  return self._read() if self._is_read else self._write()


Progress:   0%|          | 0/6 [00:00<?, ?it/s]

/nsls2/conda/envs/2025-2.0-py311-tiled/lib/python3.11/site-packages/pymongo/synchronous/mongo_client.py:2554: UserWarning: MongoClient opened before fork. May not be entirely fork-safe, proceed with caution. See PyMongo's documentation for details: https://www.mongodb.com/docs/languages/python/pymongo-driver/current/faq/#is-pymongo-fork-safe-
  return self._read() if self._is_read else self._write()
/nsls2/conda/envs/2025-2.0-py311-tiled/lib/python3.11/site-packages/pymongo/synchronous/mongo_client.py:2554: UserWarning: MongoClient opened before fork. May not be entirely fork-safe, proceed with caution. See PyMongo's documentation for details: https://www.mongodb.com/docs/languages/python/pymongo-driver/current/faq/#is-pymongo-fork-safe-
  return self._read() if self._is_read else self._write()
/nsls2/conda/envs/2025-2.0-py311-tiled/lib/python3.11/site-packages/pymongo/synchronous/mongo_client.py:2554: UserWarning: MongoClient opened before fork. May not be entirely fork-safe, proceed 

Parallel interp_sub_pix:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
### transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)

energy = params['energy']
delta = params['delta']
gamma = params['gamma']
num_angle = params['number of angles']
th_step = params['angle step']
pix = params['pixel size']
det_dist = params['detector distance']
offset = np.asarray(params['roi offset'])
data_store = params['data store']  

method = {'fit_type': 'com',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1], # number of peaks in each direction along qx, qy, and qz
          'mask': 'tot', # reference image used for reference
          'mask threshold': 0.1 # pixels with values below maximun*thresh are set to zero
         }  


# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset,stack,elem_list)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store,desired_workers=10)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
rsm.calcSTRAIN(method) 
# show results
rsm.disp()
# save results
rsm.save(output_path)
# also save the entire object
save_file = f"{output_path}all_data.obj"
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

Processing scans in parallel:   0%|          | 0/3600 [00:00<?, ?it/s]

raw det_data is deleted
qxz_data: [pos,qx,qz] with dimensions of (60, 60, 8, 619)
qyz_data: [pos,qy,qz] with dimensions of (60, 60, 99, 619)


  0%|          | 0/3600 [00:00<?, ?it/s]

In [9]:
rsm.run_interactive(scale='log')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 4 dimensions. The detected shape was (2, 60, 60, 619) + inhomogeneous part.

In [ ]:
qz = np.sum(rsm.qxz_data,-2)
plt.figure()
 
x = rsm.zq[0,0,:]
y = qz[35,35,:]
popt, fit_y = fit_peaks(x, y,peak_type='gaussian',n_peaks=3)
plt.plot(x,y,'black',x,fit_y,'red')
plt.show(block=False)

In [ ]:
method = {'fit_type': 'peak',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1],
          'mask': 'tot',
          'mask threshold': 0.1
         } # number of peaks in each direction, qx, qy, and qz

rsm.calcSTRAIN(method) 
# show results
rsm.disp()

In [ ]:
t = np.zeros_like(rsm.fluo_stack[0])
maxV = np.max(rsm.tot)
t[np.argwhere(rsm.tot>maxV*0.1)] = 1
plt.imshow(rsm.strain*t)
plt.show(block=False)

In [ ]:
params = {"scan ids": sid_list,
          "fluorescence data path": data_path,
          "output path": output_path,
          "element list": elem_list,
          "element for alignment": elem,
          "alignment matrix": trans_matrix,
          "scan dimensions": [scan_row,scan_col],
          "detector name": "merlin1",
          "detector roi": roi,
          "threshold": None,
          "monitor": "sclr1_ch4",
          "energy": energy,
          "gamma": gamma,
          "delta": delta,
          "number of angles": num_angle,
          "angle step": th_step,
          "pixel size": pix,
          "roi offset": offset,
          "data store": "reduced",
          "detector distance": det_dist,
         }
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created directory: {output_path}")

def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy(i) for i in obj]
    else:
        return obj
params_cleaned = convert_numpy(params)
    
with open(f"{output_path}parameters.json", "w") as f:
    json.dump(params_cleaned, f, indent=4)
    print(f"write parameters to {output_path}parameters.json")